# WEEK 1

### Importing packages and libraries

In [1]:
! pip install folium==0.5.0
! pip install geopy
! pip install bs4

     |████████████████████████████████| 79 kB 7.5 MB/s  eta 0:00:01
  Created wheel for folium: filename=folium-0.5.0-py3-none-any.whl size=76240 sha256=fd6e42ed33dee9f5fb0c3c39b4efcf7414b2ce8623bd6016645611c9bee0e368
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/b2/2f/2c/109e446b990d663ea5ce9b078b5e7c1a9c45cca91f377080f8
Successfully built folium
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1272 sha256=7159966f5a845614780dbfd9b6ee5d1aa4d2e0cb1b5ae05518d5cfe164c55027
  Stored in directory: /tmp/wsuser/.cache/pip/wheels/0a/9e/ba/20e5bbc1afef3a491f0b3bb74d508f99403aabe76eda2167ca
Successfully built bs4


In [2]:
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
from urllib.request import urlopen

print('Libraries imported.')

Libraries imported.


In [16]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
htlm_doc = urlopen(url).read().decode('utf-8') 
soup = BeautifulSoup(htlm_doc, 'html.parser') # Navigating the tree, BeautifulSoup object

wiki = soup.body.table.tbody

### extracting all the URLs data found within the wikipedia page into the data frame

In [14]:
def get_cell(element): # extracting all the URLs found within the wikipedia page
    cells = element.find_all('td') # need to get all the <td> tags
    x = []
    
    for cell in cells:
        if cell.a: # Using a tag name as an attribute, index            
            if (cell.a.text): 
                x.append(cell.a.text)
                continue
        x.append(cell.string.strip())
        
    return x

In [17]:
def get_row():    
    data = []  
    
    for tr in wiki.find_all('tr'): #find text content
        x = get_cell(tr)
        if len(x) != 3:
            continue
        data.append(x) #append the extract data from wikidepia into our data frame
    
    return data

In [6]:
data = get_row()
columns = ['Postcode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(data, columns=columns) # create the pandas data frame with the given columns
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [7]:
df.shape

(180, 3)

### Data Cleaning

In [8]:
df1 = df[df.Borough != 'Not assigned'] # Ignore cells with a borough that is Not assigned
df1 = df1.sort_values(by=['Postcode','Borough']) 

df1.reset_index(inplace=True) # new index
df1.drop('index',axis=1,inplace=True) # drop former index

df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [11]:
df_postal = df1['Postcode']
df_postal.drop_duplicates(inplace=True) # drop same postal codes
df2 = pd.DataFrame(df_postcodes)
df2['Borough'] = '';
df2['Neighbourhood'] = ''; # create an new clean data frame


df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df1.reset_index(inplace=True)
df1.drop('index', axis=1, inplace=True)

for i in df2.index: # same postal code neighborhood two rows will be combined into one row with the neighborhoods separated with a comma
    for j in df1.index:
        if df2.iloc[i, 0] == df1.iloc[j, 0]:
            df2.iloc[i, 1] = df1.iloc[j, 1]
            df2.iloc[i, 2] = df2.iloc[i, 2] + ',' + df1.iloc[j, 2]
            
for i in df2.index: # a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    s = df2.iloc[i, 2] # # a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.
    if s[0] == ',':
        s =s [1:]
    df2.iloc[i,2 ] = s

In [12]:
df2.reset_index(inplace=True)
df2.drop('index', axis=1, inplace=True)
df2.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
df2.shape

(103, 3)